#  Набо функций, для работы с системой ЕМИСС

###  Автор: Кирилл Казанцев

In [1]:
from selenium import webdriver
from time import sleep
import ipywidgets as widgets
import ipython_blocking
ipython_blocking.load_ipython_extensions()

## Ключевые функции

In [18]:
'''
to_panel_data - функция, которая преобразует данные на сайте ЕМИСС в панельный формат. Принимает аргументы: browser - webdriver 
объект. varible - если True (default значение), то функция преполагает, что переменная уже содержится в стобцах. Если нет, то надо 
указать часть названия переменной, чтобы передвинуть ее из строки в столбик

choose_all_data - функция, которой проходит по всем фильтрам и выбирает все возможные категории переменных

download_data -  скачивает файл

find_all_filters - находит все актуальные фильтры на страницу
'''

def to_panel_data(browser, variable=True):
    
    from selenium.webdriver import ActionChains

    browser.execute_script("arguments[0].click();", browser.find_elements_by_xpath("//*[contains(text(), 'Настройки')]")[0])
    sleep(1)
    
    target_field = browser.find_element_by_xpath('//*[@id="agrid-settings-block"]/div[4]/table/tbody/tr[2]/td[1]/div')
    element = browser.find_elements_by_xpath("//*[contains(text(), 'Год')]")
    ActionChains(browser).drag_and_drop(element[0], target_field).perform() 
    sleep(1)
    
    element = browser.find_elements_by_xpath("//*[contains(text(), 'Период')]")
    ActionChains(browser).drag_and_drop(element[0], target_field).perform() 
    sleep(2)
    
    target_field = browser.find_element_by_xpath('//*[@id="agrid-settings-block"]/div[4]/table/tbody/tr[1]/td[2]/div')
    element = browser.find_elements_by_xpath("//*[contains(text(), 'Показатель')]")
    ActionChains(browser).drag_and_drop(element[0], target_field).perform() 
    sleep(1)
    
    if variable is not True:
        
        target_field = browser.find_element_by_xpath('//*[@id="agrid-settings-block"]/div[4]/table/tbody/tr[1]/td[2]/div')
        element = browser.find_elements_by_xpath("//*[contains(text(), '" + variable+"')]")
        ActionChains(browser).drag_and_drop(element[0], target_field).perform() 
    
    browser.execute_script("arguments[0].click();", browser.find_elements_by_xpath("//*[contains(text(), 'Обновить')]")[0])
    
    
def choose_all_data(browser):

    filters = browser.find_elements_by_class_name("k-filter")
    for fltr in filters:
        browser.execute_script("arguments[0].click();", fltr)
    
    buttons = browser.find_elements_by_xpath("//*[contains(text(), 'Очистить')]")
    for button in buttons:
        browser.execute_script("arguments[0].click();", button)
    
    buttons = browser.find_elements_by_xpath("//*[contains(text(), 'Выбрать все')]")
    for button in buttons:
        browser.execute_script("arguments[0].click();", button)
        
def download_data(browser):
    browser.execute_script("arguments[0].click();", browser.find_element_by_id("download_excel_file"))
    
def input_observe(ev):
    value = ev['new']
    if len(value) >= 1:
        button.disabled = False
        button.button_style = 'success'
    else:
        button.disabled = True
        button.button_style = ''
        
        
def find_all_filters(browser):
    filters = browser.find_elements_by_class_name("k-filter")
    
    for el in filters:
        browser.execute_script("arguments[0].click();", el)

    item_list = browser.find_elements_by_class_name("k-other-filters")
    
    browser.execute_script("arguments[0].click();", browser.find_elements_by_xpath("//*[contains(text(), 'Фильтровать')]")[0])
    
    return(len(item_list))

## Пример использования

In [4]:
# выбираем директорию, куда будем скачивать
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "....."}
chromeOptions.add_experimental_option("prefs",prefs)

In [20]:
#  Запускам Google Chrome
browser = webdriver.Chrome(chrome_options=chromeOptions)

In [21]:
# Выбираем страницу с нужным показателем по ссылке

browser.get("https://fedstat.ru/indicator/31455")


In [ ]:
# преобразуем показатель в панельный вид

to_panel_data(browser, 'Вид начисленной')

In [27]:
#запускаем цикл, который позволит выбирать по каждому из показателей

for i in range(find_all_filters(browser)):
    sleep(2)
    filters = browser.find_elements_by_class_name("k-filter")
    sleep(1)
    browser.execute_script("arguments[0].click();", filters[i])
    sleep(1)
    browser.execute_script("arguments[0].click();", browser.find_element_by_xpath("//*[contains(text(), 'Очистить')]"))
    sleep(1)
    
    item_list = browser.find_element_by_class_name("k-other-filters")
    
    query_input = widgets.SelectMultiple(
        options=('Выбрать все\n' + item_list.text).split('\n'),
        rows=10,
        disabled=False)

    button = widgets.Button(description="Submit", disabled=True)

    query_input.observe(input_observe, 'value')
    box = widgets.VBox(children=[query_input, button])

    display(box)

    %block button

    for el in query_input.value:
        element = browser.find_elements_by_xpath("//*[contains(text(), '" + el+"')]")

        browser.execute_script("arguments[0].click();", element[-1])
        sleep(0.5)
    browser.execute_script("arguments[0].click();", browser.find_element_by_xpath("//*[contains(text(), 'Фильтровать')]"))
    sleep(1)

In [28]:
#  скачиваем отобранные данные

download_data(browser)

In [331]:
df.to_csv('final.csv')

In [330]:
df.shape

(2352, 27)

In [377]:
df.to_csv('final.csv')